In [1]:
# importing libraries

from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
import nltk    # Natural Language Toolkit library for machine learning
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer    # for lemmatizing and removing duplicate words
import json    # for our json mini dataset
import numpy as np   # for numpy arrays and functionality
import re      # for searching within a string
import os
from io import BytesIO
import random  # random choices
import pickle  # to store temporary data in a pickle file
# Tensorflow Keras Libraries for training chatbot on dataset 
from keras.models import load_model

# Libraries to create GUI
import tkinter
from tkinter import *
import speech_recognition as sr
from gtts import gTTS
import pyttsx3

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [3]:
def cleanSentences(sentence):
    sentences = nltk.word_tokenize(sentence)
    sentences = [lemmatizer.lemmatize(word.lower()) for word in sentences]
    
    return sentences

In [4]:
def bow(sentence, words, show_details=True):
    sentences = cleanSentences(sentence)
    basket = [0] * len(words)
    
    for sentence in sentences:
        for i, j in enumerate(words):
            if j == sentence:
                basket[i] = 1
                
                if show_details:
                    print("Found in basket:", j)
    return np.array(basket)

In [5]:
def predictClass(sentence, model):
    predict = bow(sentence, words, show_details=False)
    res = model.predict(np.array([predict]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, j] for i, j in enumerate(res) if j > ERROR_THRESHOLD]
    results.sort(key=lambda x:x[1], reverse=True)
    returned = []
    
    for result in results:
        returned.append({"intent": classes[result[0]], "probability": str(result[1])})
    return returned

In [6]:
chatbot = ChatBot("Personi Coder")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
bot_template = "BOT: {0}"
user_template = "{0}: {1}"
user_name = "Zaid"
weather = ["cloudy", "sunny", "Rainy", "Cold", "Hot"]
responses = {
    "What's Your name?" : [
        "My name is: {0}".format(user_name),
        "They call me {0}".format(user_name),
        "I go by {0}".format(user_name)
    ],
    "What's today's Weather?" : [
        "The weather is {0}".format(random.choice(weather)),
        "It's {0} today.".format(random.choice(weather))
    ],
    "default" : ["Default Message"]
}

responses2 = {
            'question': ["I don't know :(", 'you tell me!'],
            'statement': ['tell me more!',
            'why do you think that?',
            'how long have you felt this way?',
            'I find that extremely interesting',
            'can you back that up?',
            'oh wow!',
            ':)'
    ]
}

rules = {'I want (.*)': [
                         'What would it mean if you got {0}',
                         'Why do you want {0}',
                         "What's stopping you from getting {0}"
                        ],
         'do you remember (.*)': [
                                  'Did you think I would forget {0}',
                                  "Why haven't you been able to forget {0}",
                                  'What about {0}',
                                  'Yes .. and?'
                                 ],
         'do you think (.*)': [
                               'if {0}? Absolutely.', 'No chance'
                              ],
         'if (.*)': [
                     "Do you really think it's likely that {0}",
                     'Do you wish that {0}',
                     'What do you think about {0}',
                     'Really--if {0}'
                    ]
        }

keywords = {'greet': ['hello', 'hi', 'hey'], 'goodbye': ['bye', 'farewell'], 'thankyou': ['thank', 'thx']}

In [8]:
def replacePronouns(message):
    message = message.lower()
    
    if 'me' in message:
        return re.sub("me", "you", message)
    if 'my' in message:
        return re.sub("my", "your", message)
    if 'your' in message:
        return re.sub("your", "my", message)
    if 'you' in message:
        return re.sub("you", "me", message)
    
    return message

In [9]:
def matchRule(rules, message):
    response, phrase = "default", None
    
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        
        if match is not None:
            response = random.choice(responses)
            
            if '{0}' in response:
                phrase = match.group(1)
    
    return response.format(phrase), phrase

In [10]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [11]:
# def respond(message):
#     #bot_message = "You said: " + message
    
# #     if message in responses:
# #         bot_message = random.choice(responses[message])
# #     else:
# #         bot_message = random.choice(responses["default"])
# #     if message.endswith("?"):
# #         return random.choice(responses2['question'])
# #     return random.choice(responses2['statement'])
#     response, phrase = matchRule(rules, message)
    
#     if '{0}' in response:
#         print(phrase)
#         phrase = replacePronouns(phrase)
#         response = response.format(phrase)
#     return response

def respond(ints, intents):
    print(ints)
    tag = ints[0]['intent']
    total_intents = intents['intents']
    
    for intent in total_intents:
        if intent['tag'] == tag:
            result = random.choice(intent['responses'])
            break
    return result

def chatbotResponse(text):
    check = random.randint(1, 2)
    response = chatbot.get_response(text)
    language = "en"
    speak(response)
#     obj = gTTS(text=text, lang=language, slow=False)
#     obj.save("audio.mp3")
#     os.system("mpg321 audio.mp3")
#     print(response)
    
#     if check == 1:
#         ints = predictClass(text, model)
#         response = respond(ints, intents)
#         print("here")
#     elif check == 2:
#         response = chatbot.get_response(text)
#         print("here2")
    
    return response

In [12]:
Questions = [
    """Question 1:
    A. Do you expand Energy, and enjoy Social Groups or
    B. Conserve Energy, and Enjoy One-on-One conversations
    """,
    """Question 2:
    A. Are you More Outgoing, and Think Out Loud Person or
    B. Are you more reserved, and Think to yourself person
    """,
    """Question 3:
    A. Do you Seek many tasks, public activities and Interaction with others or
    B. Seek Privacy, and Solitary activities with Quiet to concentrate
    """,
    """Question 4:
    A. Are you an External, Communicative and Express Yourself type of Person or
    B. an Internal, Reticent and Keep to yourself type
    """,
    """Question 5:
    A. Would you categorize yourself as Active, Initiate or
    B. Reflective, Deliberate
    """,
    """Question 6:
    A. Do you interpret words literally or
    B. Do you look for deeper meanings and possiblities
    """,
    """Question 7:
    A. Are you more Practical, Realistic, Experimental or
    B. Imaginative, Innovative, Theoretical
    """,
    """Question 8:
    A. Do these words mean more to you: Standard, Usual, Conventional or
    B. Different, Unique and Novel
    """,
    """Question 9:
    A. Would you focus more on here-and-now or
    B. Look for the big picture, global perspective and to the future
    """,
    """Question 10:
    A. Do you like facts, things and What is or
    B. Ideas, Dreams, What could be and Philosophy
    """,
    """Question 11:
    A. Would you categorize yourself as Logical, Thinking, and Questioning everything or
    B. Empathetic, Feeling and Accomodating
    """,
    """Question 12:
    A. Are you more Candid, Straightforward, and frank or
    B. Tactful, Kind and Encouraging
    """,
    """Question 13:
    A. Are you more firm, tend to criticize everything and hold the line or
    B. Gentle, Tend to appreciate and Conciliate
    """,
    """Question 14:
    A. Are you tough minded and just or
    B. Tender-hearted and Merciful
    """,
    """Question 15:
    A. Are you a matter-of-fact, Issue-Oriented person or
    B. Sensitive, People-oriented and Compassionate
    """,
    """Question 16:
    A. Would you say you are more organized and Orderly or
    B. Flexible and Adaptable
    """,
    """Question 17:
    A. Do you plan and schedule events or
    B. Are you more unplanned and spontaneous
    """,
    """Question 18:
    A. Are you more Regular and Structured or
    B. Easygoing and Live and Let Live kind of person
    """,
    """Question 19:
    A. Do you prepare and Plan ahead or
    B. Go with the Flow and Adopt as you go
    """,
    """Question 20:
    A. Do you like Control and Governance or
    B. Latitude and Freedom"""
]

test_started = False

In [13]:
Questions_started = False
key = 0
count_of_a = 0
count_of_b = 0
dichotomy = ""
count = 0
def SixteenPersonalityTest(started, key, msg):
    global Questions_started
    if not Questions_started:
        ChatLog.insert(END, bot_template.format("Please Answer Questions in the form of A or B\n\n"))
        speak("Please Answer Questions in the form of A or B\n\n")
        ChatLog.config(foreground="#442265", font=("Times New Roman", 12))
        Questions_started = True
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, bot_template.format(Questions[key]) + "\n\n")
    speak(Questions[key])
    ChatLog.config(foreground="#442265", font=("Times New Roman", 12))
    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

In [14]:
# def sendMessage(message):
#     print(user_template.format(user_name, message))
    
#     response = respond(message)
    
#     print(bot_template.format(response))
    
# sendMessage("do you remember my last birthday")

def sendMessage():
    global key
    global test_started
    global count_of_a
    global count_of_b
    global count
    global dichotomy
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, user_template.format(user_name, msg) + "\n\n")
        ChatLog.config(foreground="#442265", font=("Times New Roman", 12))
        
        if "take 16 personalities test" in msg:
            SixteenPersonalityTest(Questions_started, key, msg)
            test_started = True
            key += 1
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        elif test_started:
            if key < 20:
                SixteenPersonalityTest(Questions_started, key, msg)
            
                answer = ''
                if not (answer == 'A' or answer == 'B'):
                    count_of_a = 0
                    count_of_b = 0

                    try:
                        answer = msg.upper()
                        if not (answer == 'A' or answer == 'B'):
                            raise ValueError("Invalid Input")
                            base.destroy()
                    except ValueError as error:
                            print(error)
                            base.destroy()
                    else:
                        if answer == 'A':
                            count_of_a += 1
                        if answer == 'B':
                            count_of_b += 1
                        count += 1

            if count == 5:
                if count_of_a > count_of_b:
                    dichotomy = dichotomy + "E"
                else:
                    dichotomy = dichotomy + "I"
            else:
                if count == 10:
                    if count_of_a > count_of_b:
                        dichotomy = dichotomy + "S"
                    else:
                        dichotomy = dichotomy + "N"
                else:
                    if count == 15:
                        if count_of_a > count_of_b:
                            dichotomy = dichotomy + "T"
                        else:
                            dichotomy = dichotomy + "F"
            key += 1
            if key == 21:
                if count_of_a > count_of_b:
                    dichotomy = dichotomy + "J"
                else:
                    dichotomy = dichotomy + "P"
                f = open("Personality Report.txt", "a+")
                f.write("Name: " + user_name + "\n")
                f.write("Your personality type is: " + dichotomy + "\n")
                f.close()
                print(dichotomy)
                base.destroy()
            else:
                ChatLog.config(state=DISABLED)
                ChatLog.yview(END)
        else:
            res = chatbotResponse(msg)
            ChatLog.insert(END, bot_template.format(res) + "\n\n")
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)

In [15]:
base = Tk()
base.title("Welcome to Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

sendButton = Button(base, font=("Times New Roman", 12, 'bold'), text="Send", width="12", height=5, bd = 0, bg = "#32de97",
                    activebackground="#3c9d9b", fg="#ffffff", command=sendMessage)

EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
sendButton.place(x=6, y=401, height=90)

base.mainloop()

INFP


In [ ]:
replacePronouns("my last birthday")